In [1]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_curve, auc, classification_report
from xgboost import XGBClassifier

In [2]:
train_df = pd.read_csv("../../../dataset/open_world/openworld_train.csv")
test_df  = pd.read_csv("../../../dataset/open_world/openworld_test.csv")

In [3]:
# feature/label 분리
target_col = train_df.columns[-1]   # 마지막 컬럼이 label

# open-world label 통합
train_df[target_col] = train_df[target_col].apply(lambda x: 0 if x == 95 else 1)
test_df[target_col]  = test_df[target_col].apply(lambda x: 0 if x == 95 else 1)

X_train = train_df.drop(columns=[target_col]).values
y_train = train_df[target_col].astype(np.float32).values
X_test  = test_df.drop(columns=[target_col]).values
y_test  = test_df[target_col].astype(np.float32).values

In [4]:
# Best MLP param.
best_mlp_params = {
    "hidden_layer_sizes": (288, 80),
    "alpha": 3.813848083517318e-05,
    "learning_rate_init": 0.0017053755428803074,
    "batch_size": 128,
    "activation": "relu",
    "solver": "adam",
}

# Train MLP ensemble (seeds: 0-4)
seeds = [0, 1, 2, 3, 4]
mlp_models = []

print("\nTraining 5 MLP models...")
for s in seeds:
    mlp = MLPClassifier(
        **best_mlp_params,
        random_state=s,
        max_iter=50
    )
    mlp.fit(X_train, y_train)
    mlp_models.append(mlp)
    print(f"MLP model seed={s} done.")


Training 5 MLP models...


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP model seed=0 done.


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP model seed=1 done.


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP model seed=2 done.


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP model seed=3 done.
MLP model seed=4 done.


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [5]:
# Train XGBoost Model
xgb_params = {
    "n_estimators": 300,
    "max_depth": 10,
    "learning_rate": 0.1890408241426905,
    "min_child_weight": 1,
    "subsample": 0.8887320375513359,
    "colsample_bytree": 0.9912853863634461,
    "gamma": 0.014498503088113818,
    "reg_lambda": 2.7498279931612504,
    "reg_alpha": 0.0985171472007404,
    "random_state": 42,
}

print("\nTraining XGBoost model (fill your own params in xgb_params)...")
xgb_model = XGBClassifier(
    **xgb_params,
    objective="binary:logistic",
    eval_metric="logloss",
    use_label_encoder=False,
)

xgb_model.fit(X_train, y_train)
print("\nTraining XGBoost model (fill your own params in xgb_params)...")
xgb_model = XGBClassifier(
    **xgb_params,
    objective="binary:logistic",
    eval_metric="logloss",
    use_label_encoder=False,
)

xgb_model.fit(X_train, y_train)
print("XGBoost training done.")

/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [18:36:09] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1724807611129/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Training XGBoost model (fill your own params in xgb_params)...

Training XGBoost model (fill your own params in xgb_params)...
XGBoost training done.


In [6]:
# Soft-Voting Ensemble (MLP x5 + XGBoost)
print("\nEnsembling predictions...")

proba_list = []

# MLP 5개 확률
for model in mlp_models:
    p = model.predict_proba(X_test)[:, 1]
    proba_list.append(p)

# XGBoost 확률
xgb_proba = xgb_model.predict_proba(X_test)[:, 1]
proba_list.append(xgb_proba)

# 평균 확률 (soft voting)
proba_ensemble = np.mean(proba_list, axis=0)
y_pred = (proba_ensemble >= 0.5).astype(int)


Ensembling predictions...


In [7]:
# Evaluation
test_acc = accuracy_score(y_test, y_pred)
test_f1_macro = f1_score(y_test, y_pred, average="macro")
test_f1_micro = f1_score(y_test, y_pred, average="micro")
test_f1_weighted = f1_score(y_test, y_pred, average="weighted")

# ROC-AUC
test_roc_auc = roc_auc_score(y_test, proba_ensemble)

# PR-AUC
prec, rec, _ = precision_recall_curve(y_test, proba_ensemble)
test_pr_auc = auc(rec, prec)

print("\n========== [TEST RESULTS - MLP5 + XGBoost Ensemble] ==========")
print(f"Accuracy        : {test_acc:.4f}")
print(f"F1 (macro)      : {test_f1_macro:.4f}")
print(f"F1 (micro)      : {test_f1_micro:.4f}")
print(f"F1 (weighted)   : {test_f1_weighted:.4f}")
print(f"ROC-AUC         : {test_roc_auc:.4f}")
print(f"PR-AUC          : {test_pr_auc:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))


========== [TEST RESULTS - MLP5 + XGBoost Ensemble] ==========
Accuracy        : 0.8980
F1 (macro)      : 0.8851
F1 (micro)      : 0.8980
F1 (weighted)   : 0.8971
ROC-AUC         : 0.9579
PR-AUC          : 0.9761

Classification Report:
              precision    recall  f1-score   support

         0.0     0.8796    0.8160    0.8466      3000
         1.0     0.9067    0.9412    0.9236      5700

    accuracy                         0.8980      8700
   macro avg     0.8932    0.8786    0.8851      8700
weighted avg     0.8974    0.8980    0.8971      8700

